
### Imports

In [94]:
%load_ext cython
%load_ext cythonmagic
%matplotlib inline
%matplotlib notebook

import pandas as pd
import numpy as np
import re, reprlib, sys
import matplotlib.pyplot as plt
from itertools import *
import random as ran
from logging import *
from pprint import *
from time import *
import shlex, subprocess

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

from scipy.cluster.vq import *

import nltk as n
import nltk, nltk.classify.util, nltk.metrics, nltk.tokenize, nltk.stem
from nltk.corpus import *
from nltk.stem import *
from nltk.classify import *
from nltk.collocations import *
from nltk.metrics import *
from nltk.probability import *
from nltk.classify.scikitlearn import *
from nltk.tag.sequential import *
from nltk.tag import *
from nltk.tag.util import *
# n.download()

from sklearn_pandas import *

import sklearn as sk
from sklearn import *
from sklearn.feature_extraction.text import *
from sklearn.svm import *
from sklearn.preprocessing import *
from sklearn.cross_validation import *
from sklearn.pipeline import *
from sklearn.multiclass import *
from sklearn.datasets import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.feature_selection import *
from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.tree import *
from sklearn.grid_search import *
from sklearn.base import *
from sklearn.datasets.twenty_newsgroups import *
from sklearn.decomposition import *
from sklearn.feature_extraction import *
from sklearn.metrics import *
from sklearn.utils import *

sk.utils.check_random_state(5125)
ran.seed(5125)

The cython extension is already loaded. To reload it, use:
  %reload_ext cython
The cythonmagic extension is already loaded. To reload it, use:
  %reload_ext cythonmagic


### R script adding new features to dataset 

In [95]:
# http://www.mango-solutions.com/wp/2015/10/integrating-python-and-r-part-ii-executing-r-from-python-and-vice-versa/
command = 'Rscript'
path2script = 'insert_features.R'

# Build subprocess command
cmd = [command, path2script]

subprocess.check_output(cmd, universal_newlines=True)

''

In [96]:
data_n = pd.read_csv("../raw_data/data_n.csv", parse_dates=True, infer_datetime_format=True)
data_n.head()

,Unnamed: 0,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,...,Number_of_Dots,Number_of_Commas,Number_of_Semicolons,Number_of_Colons,Average_Word_Length,Lexical_Diversity,Number_of_FunctionalWords,Number_of_Pronouns,Number_of_PROPNAMEs,SentimentNumeric
0,1,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,...,1,0,0,0,4.400,0.700,2,0,0,2
1,2,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3,3.15,3.25,4.4,n,y,...,1,0,0,0,3.615,0.577,10,1,0,0
2,3,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3,3.15,3.25,4.4,n,y,...,1,1,0,0,3.500,0.509,14,1,0,1
3,4,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3,3.15,3.25,4.4,n,y,...,1,0,0,0,3.667,0.611,4,0,0,2
4,5,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3,3.15,3.25,4.4,n,y,...,1,0,0,0,2.333,0.857,1,0,0,2


### Replace 'y' and 'n'   AND easy split

In [97]:
# http://stackoverflow.com/a/17702781
d = {'n': 0, 'y': 1}
data_n = data_n.replace(d)
data_n.head(n=1)

,Unnamed: 0,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,...,Number_of_Dots,Number_of_Commas,Number_of_Semicolons,Number_of_Colons,Average_Word_Length,Lexical_Diversity,Number_of_FunctionalWords,Number_of_Pronouns,Number_of_PROPNAMEs,SentimentNumeric
0,1,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,0,1,...,1,0,0,0,4.4,0.7,2,0,0,2


In [98]:
#norm:http://blog.yhat.com/posts/predicting-customer-churn-with-sklearn.html
# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# to_drop = ['State','Area Code','Phone','Churn?']
# churn_feat_space = data_n.drop(to_drop,axis=1)

In [99]:
neu = data_n[["#AUTHID","STATUS","cNEU","StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]]
ext = data_n[["#AUTHID","STATUS","cEXT", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]]
agr = data_n[["#AUTHID","STATUS","cAGR","StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]]
con = data_n[["#AUTHID","STATUS","cCON","StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]]
opn = data_n[["#AUTHID","STATUS","cOPN", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]]

### Split stra. k-fold

In [100]:
# neu = data_n[["#AUTHID","STATUS","cNEU"]]
# train_neu, test_neu, y_train, y_test = sk.cross_validation.train_test_split(neu, neu["cNEU"], train_size = 0.66, stratify= neu["cNEU"])
# train_neu, test_neu, y_train, y_test 

In [113]:
train_feat_neu, test_feat_neu, train_class_neu, test_class_neu = sk.cross_validation.train_test_split(neu, neu["cNEU"], train_size = 0.66, stratify = neu["cNEU"],random_state= 5152)
print(train_feat_neu.shape, test_feat_neu.shape,train_class_neu.shape ,test_class_neu.shape)

neu_X = train_feat_neu.append(test_feat_neu)
neu_Y = train_class_neu.append(test_class_neu)

train_feat_ext, test_feat_ext, train_class_ext, test_class_ext = sk.cross_validation.train_test_split(ext, ext["cEXT"], train_size = 0.66, stratify = ext["cEXT"],random_state= 5152)

ext_X = train_feat_ext.append(test_feat_ext)
ext_Y = train_class_ext.append(test_class_ext)

train_feat_agr, test_feat_agr, train_class_agr, test_class_agr = sk.cross_validation.train_test_split(agr, agr["cAGR"], train_size = 0.66, stratify = agr["cAGR"],random_state= 5152)
train_feat_con, test_feat_con, train_class_con, test_class_con = sk.cross_validation.train_test_split(con, con["cCON"], train_size = 0.66, stratify = con["cCON"],random_state= 5152)
train_feat_opn, test_feat_opn, train_class_opn, test_class_opn = sk.cross_validation.train_test_split(opn, opn["cOPN"], train_size = 0.66, stratify = opn["cOPN"],random_state= 5152)

(6545, 15) (3372, 15) (6545,) (3372,)


In [102]:
# mku = sk.pipeline.make_pipeline(SGDClassifier(), LinearSVC(),
#                                # BernoulliNB(), #MultinomialNB()
#                                # KNeighborsClassifier(),
#               RandomForestClassifier(n_jobs=-1),AdaBoostClassifier()) 

### Types of classifiers 
with their configuration

In [114]:
nick_names = ["tt SVC", "tt Linear SVC", "tt MultinomiaNB", "tt BernoulliNB", "ttRF", "ttAD", "ttKNN"]
classifiers = [
    SVC(cache_size=500, kernel = "rbf", decision_function_shape = "ovr", random_state = 5152),
    LinearSVC(random_state = 5152),
    MultinomialNB(),
    BernoulliNB(),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, n_jobs=-1, random_state = 5152),
    AdaBoostClassifier(random_state = 5152),
    KNeighborsClassifier()]

### Train + Test NEU

In [104]:
# print(classifiers, nick_names)
for name, clf in zip(nick_names, classifiers):
    clf.fit(train_feat_neu[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]], train_class_neu)
    y_pred_neu = clf.predict(test_feat_neu[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]])
    print("\nFOR:", name)
    print("Accuracy score: ", sk.metrics.accuracy_score(test_class_neu, y_pred_neu))
    print(sk.metrics.classification_report(test_class_neu, y_pred_neu))


FOR: tt SVC
Accuracy score:  0.608540925267
MAR:  0.391459074733
             precision    recall  f1-score   support

          0       0.62      0.94      0.75      2108
          1       0.35      0.05      0.09      1264

avg / total       0.52      0.61      0.50      3372


FOR: tt Linear SVC
Accuracy score:  0.379003558719
MAR:  0.620996441281
             precision    recall  f1-score   support

          0       0.57      0.03      0.05      2108
          1       0.37      0.97      0.54      1264

avg / total       0.50      0.38      0.23      3372


FOR: tt MultinomiaNB
Accuracy score:  0.624851720047
MAR:  0.375148279953
             precision    recall  f1-score   support

          0       0.63      0.99      0.77      2108
          1       0.48      0.01      0.02      1264

avg / total       0.57      0.62      0.49      3372


FOR: tt BernoulliNB
Accuracy score:  0.624555160142
MAR:  0.375444839858
             precision    recall  f1-score   support

          0  

In [105]:
eclf = VotingClassifier(estimators=[
        ("svc", ttSVC),
        ("linear_svc", ttLinearSVC)], voting='hard')


for clf, label in zip([ttSVC, ttLinearSVC], ["SVC", "LSVC"]):
    scores = cross_validation.cross_val_score(clf, neu_X[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]], neu_Y, cv=skf, scoring='accuracy')
    print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std()*2, label))
    print(scores)

NameError: name 'ttSVC' is not defined

### Train + Test EXT

In [112]:
skf = StratifiedKFold(ext["cEXT"], n_folds=10, shuffle=True, random_state = 5152)
print(skf) 

sklearn.cross_validation.StratifiedKFold(labels=[0 0 0 ..., 1 0 1], n_folds=10, shuffle=True, random_state=5152)


In [117]:
# print(classifiers, nick_names)
for name, clf in zip(nick_names, classifiers):
    clf.fit(train_feat_ext[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]], train_class_ext)
    y_pred_ext = clf.predict(test_feat_ext[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]])
    print("\nFOR:", name)
    print("Accuracy score: ", sk.metrics.accuracy_score(test_class_ext, y_pred_ext))
    print(sk.metrics.classification_report(test_class_ext, y_pred_ext))
    print(sk.metrics.confusion_matrix(test_class_ext, y_pred_ext))
    print(sk.metrics.f1_score(test_class_ext, y_pred_ext))
    print(sk.metrics.precision_score(test_class_ext, y_pred_ext))
    print(sk.metrics.recall_score(test_class_ext, y_pred_ext))
    
    scores = sk.cross_validation.cross_val_score(clf, ext_X[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]], ext_Y, cv=skf, scoring='accuracy')
    print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std()*2, name))
    


FOR: tt SVC
Accuracy score:  0.567615658363
             precision    recall  f1-score   support

          0       0.58      0.86      0.70      1941
          1       0.47      0.17      0.25      1431

avg / total       0.54      0.57      0.51      3372

[[1673  268]
 [1190  241]]
0.248453608247
0.47347740668
0.168413696716
Accuracy: 0.567 (+/- 0.03) [tt SVC]

FOR: tt Linear SVC
Accuracy score:  0.575622775801
             precision    recall  f1-score   support

          0       0.58      1.00      0.73      1941
          1       0.00      0.00      0.00      1431

avg / total       0.33      0.58      0.42      3372

[[1941    0]
 [1431    0]]
0.0
0.0
0.0
Accuracy: 0.497 (+/- 0.13) [tt Linear SVC]

FOR: tt MultinomiaNB
Accuracy score:  0.580367734282
             precision    recall  f1-score   support

          0       0.58      0.98      0.73      1941
          1       0.58      0.04      0.08      1431

avg / total       0.58      0.58      0.45      3372

[[1898   43]
 [

/usr/local/lib/python3.4/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.4/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.4/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### Train + Test CON

In [108]:
# print(classifiers, nick_names)
for name, clf in zip(nick_names, classifiers):
    clf.fit(train_feat_con[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]], train_class_con)
    y_pred_con = clf.predict(test_feat_con[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]])
    print("\nFOR:", name)
    print("Accuracy score: ", sk.metrics.accuracy_score(test_class_con, y_pred_con))
    print(sk.metrics.classification_report(test_class_con, y_pred_con))


FOR: tt SVC
Accuracy score:  0.534697508897
MAR:  0.465302491103
             precision    recall  f1-score   support

          0       0.55      0.71      0.62      1823
          1       0.49      0.32      0.39      1549

avg / total       0.52      0.53      0.52      3372


FOR: tt Linear SVC
Accuracy score:  0.459964412811
MAR:  0.540035587189
             precision    recall  f1-score   support

          0       0.57      0.00      0.01      1823
          1       0.46      1.00      0.63      1549

avg / total       0.52      0.46      0.29      3372


FOR: tt MultinomiaNB
Accuracy score:  0.535290628707
MAR:  0.464709371293
             precision    recall  f1-score   support

          0       0.55      0.81      0.65      1823
          1       0.49      0.21      0.30      1549

avg / total       0.52      0.54      0.49      3372


FOR: tt BernoulliNB
Accuracy score:  0.542408066429
MAR:  0.457591933571
             precision    recall  f1-score   support

          0  

### Train + Test AGR

In [109]:
# print(classifiers, nick_names)
for name, clf in zip(nick_names, classifiers):
    clf.fit(train_feat_agr[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]], train_class_agr)
    y_pred_agr = clf.predict(test_feat_agr[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]])
    print("\nFOR:", name)
    print("Accuracy score: ", sk.metrics.accuracy_score(test_class_agr, y_pred_agr))
    print(sk.metrics.classification_report(test_class_agr, y_pred_agr))


FOR: tt SVC
Accuracy score:  0.521352313167
MAR:  0.478647686833
             precision    recall  f1-score   support

          0       0.49      0.34      0.40      1581
          1       0.54      0.68      0.60      1791

avg / total       0.51      0.52      0.51      3372


FOR: tt Linear SVC
Accuracy score:  0.531731909846
MAR:  0.468268090154
             precision    recall  f1-score   support

          0       0.54      0.01      0.02      1581
          1       0.53      0.99      0.69      1791

avg / total       0.53      0.53      0.38      3372


FOR: tt MultinomiaNB
Accuracy score:  0.540925266904
MAR:  0.459074733096
             precision    recall  f1-score   support

          0       0.54      0.16      0.24      1581
          1       0.54      0.88      0.67      1791

avg / total       0.54      0.54      0.47      3372


FOR: tt BernoulliNB
Accuracy score:  0.548042704626
MAR:  0.451957295374
             precision    recall  f1-score   support

          0  

### Train + Test OPN

In [110]:
# print(classifiers, nick_names)
for name, clf in zip(nick_names, classifiers):
    clf.fit(train_feat_opn[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]], train_class_opn)
    y_pred_opn = clf.predict(test_feat_opn[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity", "Number_of_FunctionalWords", "Number_of_Pronouns", "Number_of_PROPNAMEs", "SentimentNumeric"]])
    print("\nFOR:", name)
    print("Accuracy score: ", sk.metrics.accuracy_score(test_class_opn, y_pred_opn))
    print(sk.metrics.classification_report(test_class_opn, y_pred_opn))


FOR: tt SVC
Accuracy score:  0.742586002372
MAR:  0.257413997628
             precision    recall  f1-score   support

          0       0.42      0.01      0.01       866
          1       0.74      1.00      0.85      2506

avg / total       0.66      0.74      0.64      3372


FOR: tt Linear SVC
Accuracy score:  0.743179122183
MAR:  0.256820877817
             precision    recall  f1-score   support

          0       0.00      0.00      0.00       866
          1       0.74      1.00      0.85      2506

avg / total       0.55      0.74      0.63      3372


FOR: tt MultinomiaNB
Accuracy score:  0.744661921708
MAR:  0.255338078292
             precision    recall  f1-score   support

          0       0.62      0.02      0.03       866
          1       0.75      1.00      0.85      2506

avg / total       0.71      0.74      0.64      3372


FOR: tt BernoulliNB
Accuracy score:  0.743179122183
MAR:  0.256820877817
             precision    recall  f1-score   support

          0  

/usr/local/lib/python3.4/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
